In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download()

In [ ]:
import os
os.chdir(os.path.expanduser("~"))
os.chdir('/content/gdrive/MyDrive/ML/imdb')

In [ ]:
import numpy as np

class MultinomialNaiveBayes:
    def __init__(self, nb_classes, nb_words, alpha):
        self.nb_classes = nb_classes
        self.nb_words = nb_words
        self.alpha = alpha
  
    def fit(self, X, Y):
        nb_examples = X.shape[0]

        # Racunamo P(Klasa) - priors
        # np.bincount nam za datu listu vraca broj pojavljivanja svakog celog
        # broja u intervalu [0, maksimalni broj u listi]
        self.priors = np.bincount(Y) / nb_examples
        print('Priors:')
        print(self.priors)

        # Racunamo broj pojavljivanja svake reci u svakoj klasi
        occs = np.zeros((self.nb_classes, self.nb_words))
        
        for i in range(nb_examples):
            c = Y[i]
            for w in range(self.nb_words):
                cnt = X[i][w]
                occs[c][w] += cnt
        print('Occurences:')
        print(occs)
        
        # Racunamo P(Rec_i|Klasa) - likelihoods
        self.like = np.zeros((self.nb_classes, self.nb_words))
        for c in range(self.nb_classes):
            for w in range(self.nb_words):
                up = occs[c][w] + self.alpha
                down = np.sum(occs[c]) + self.nb_words*self.alpha
                self.like[c][w] = up / down
        print('Likelihoods:')
        print(self.like)
          
    def predict(self, bow):
        # Racunamo P(Klasa|bow) za svaku klasu
        probs = np.zeros(self.nb_classes) 
        for c in range(self.nb_classes):
            prob = np.log(self.priors[c])
            for w in range(self.nb_words):
                cnt = bow[w]
                prob += cnt * np.log(self.like[c][w])
            probs[c] = prob
        # Trazimo klasu sa najvecom verovatnocom
        prediction = np.argmax(probs)
        return prediction
  

In [ ]:
from nltk import FreqDist
from nltk.tokenize import wordpunct_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math

def numocc_score(word, doc):
  return doc.count(word)

score_fn = numocc_score

# Trenutni direktorijum (pos, neg)
dataset = os.listdir()


negative = dataset[0]
positive = dataset[1]
corpus_positive = []
corpus_negative = []
clean_corpus_positive = []
clean_corpus_negative = []
pos_cnt = 0
neg_cnt = 0
cnt = 0

# Ucitavanje fajlova
for folder in os.listdir():
    
    for filename in os.listdir(folder):
        fdir = os.path.join(folder, filename)
        if(os.path.isfile(fdir) and fdir.endswith('.txt')):
            f = open(fdir, 'r')
            s = f.read() 
            if folder == 'pos':
                corpus_positive.append(s)
                pos_cnt += 1
            elif folder == 'neg':
                corpus_negative.append(s)
                neg_cnt += 1
            f.close()

dataset = [corpus_positive, corpus_negative]

for i, corpus in enumerate(dataset):
    
    # Cistimo korpus
    clean_corpus = []
    stemmer = PorterStemmer()
    stop_punc = set(stopwords.words('english')).union(set(punctuation))
    for doc in corpus:
        words = wordpunct_tokenize(doc)
        words_lower = [w.lower() for w in words]
        words_filtered = [w for w in words_lower if w not in stop_punc]
        words_stemmed = [stemmer.stem(w) for w in words_filtered]
        clean_corpus.append(words_stemmed)
    if i == 0:
        clean_corpus_positive = clean_corpus
    elif i == 1:
        clean_corpus_negative = clean_corpus


# Pravimo vokabular
vocab_set = set()
for clean_corpus in [clean_corpus_positive, clean_corpus_negative]:
    for doc in clean_corpus:
        for word in doc:
            vocab_set.add(word)
vocab = list(vocab_set)

X1 = None
X2 = None

dataset = [clean_corpus_positive, clean_corpus_negative]

# Pravimo feature vektore (BoW metoda)
for i, corpus in enumerate(dataset):
    X = np.zeros((len(corpus), len(vocab)), dtype=np.float32)
    for doc_idx in range(len(corpus)):
        doc = corpus[doc_idx]
        for word_idx in range(len(vocab)):
            word = vocab[word_idx]
            cnt = score_fn(word, doc)
            X[doc_idx][word_idx] = cnt
    if i == 0:
        X1 = X
    elif i == 1:
        X2 = X

# Spajamo feature vektore / nizove targeta (pamtimo koji test primer je pozitivan/negativan)
X = np.concatenate((X1, X2), axis=0)
Y1 = [0]*pos_cnt
Y2 = [1]*neg_cnt
Y1 = np.asarray(Y1)
Y2 = np.asarray(Y2)
Y = np.concatenate((Y1,Y2), axis=0)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Testiranje
sum = 0
cor_pos = 0
cor_neg = 0
incor_pos = 0
incor_neg = 0
for k in range(3):
    print('*****************************')

    # Mapa broja pogodaka/promasaja
    vals = {'cor':0, 'incor':0}

    # Mesanje i splitovanje dataseta
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

    class_names = ['Positive', 'Negative']

    # Inicijalizacija modela
    model = MultinomialNaiveBayes(nb_classes=2, nb_words=len(vocab), alpha = 1)
    model.fit(X_train, Y_train)
    for i in range(len(X_test)):
        prediction = model.predict(X_test[i])
        if prediction == Y_test[i]:
            if prediction == 0:
                cor_pos += 1
            else:
                cor_neg += 1
            vals['cor'] += 1
        else:
            if prediction == 0:
                incor_pos += 1
            else:
                incor_neg += 1
            vals['incor'] += 1
    print(vals)
    acc = vals['cor']/(vals['cor'] + vals['incor'])
    print(round(vals['cor']/(vals['cor'] + vals['incor'])*100, 3), '% Accuracy')
    sum += acc
    conf_matrix = [[cor_neg, incor_pos], [incor_neg, cor_pos]]
    print(conf_matrix)

print('*****************************')
print(f'Average accuracy: {round(sum/3*100, 2)} %')
print('*****************************')



*****************************
Priors:
[0.508 0.492]
Occurences:
[[8. 2. 0. ... 1. 2. 0.]
 [6. 1. 3. ... 0. 0. 1.]]
Likelihoods:
[[5.77215385e-05 1.92405128e-05 6.41350427e-06 ... 1.28270085e-05
  1.92405128e-05 6.41350427e-06]
 [4.70088914e-05 1.34311118e-05 2.68622237e-05 ... 6.71555591e-06
  6.71555591e-06 1.34311118e-05]]
{'cor': 411, 'incor': 89}
82.2 % Accuracy
[[220, 46], [43, 191]]
*****************************
Priors:
[0.499 0.501]
Occurences:
[[9. 1. 0. ... 0. 1. 0.]
 [4. 1. 2. ... 0. 0. 0.]]
Likelihoods:
[[6.56749746e-05 1.31349949e-05 6.56749746e-06 ... 6.56749746e-06
  1.31349949e-05 6.56749746e-06]
 [3.26577533e-05 1.30631013e-05 1.95946520e-05 ... 6.53155066e-06
  6.53155066e-06 6.53155066e-06]]
{'cor': 399, 'incor': 101}
79.8 % Accuracy
[[424, 90], [100, 386]]
*****************************
Priors:
[0.5015 0.4985]
Occurences:
[[8. 2. 0. ... 1. 2. 0.]
 [6. 0. 3. ... 0. 0. 1.]]
Likelihoods:
[[5.75933652e-05 1.91977884e-05 6.39926280e-06 ... 1.27985256e-05
  1.91977884e-05 6

In [ ]:
# X_full_corpus = np.concatenate((clean_corpus_positive, clean_corpus_negative), axis=0)

positive_map = {}
negative_map = {}

LR_map = {}


def compare(items):
    return items[1]

for i in range(len(X)):
    for j in range(len(vocab)):
        if Y[i] == 0:
            if vocab[j] not in positive_map:
                positive_map[vocab[j]] = X[i][j]
            else:
                positive_map[vocab[j]] += X[i][j]
        else:
            if vocab[j] not in negative_map:
                negative_map[vocab[j]] = X[i][j]
            else:
                negative_map[vocab[j]] += X[i][j]


for elem in vocab:
    if positive_map[elem] >= 10 and negative_map[elem] >= 10:
        LR_map[elem] = positive_map[elem] / negative_map[elem]
        



{'perfectli': 5.5833335, 'strong': 5.2222223, 'henri': 4.6666665, 'harri': 4.6153846, 'delight': 4.5833335, 'highli': 4.571429, 'fascin': 4.1, 'excel': 4.071429, 'amaz': 4.0666666, 'uniqu': 4.0, 'friendship': 3.9, 'beauti': 3.857143, 'chemistri': 3.7272727, 'master': 3.642857, 'favorit': 3.6060605, 'marriag': 3.6, 'mari': 3.5384614, 'atmospher': 3.5263157, 'touch': 3.5172415, 'charli': 3.5, 'era': 3.5, 'perfect': 3.4473684, 'german': 3.4, 'impact': 3.3333333, 'bug': 3.2727273, 'fantast': 3.1578948, 'captur': 3.15, 'brilliant': 3.1034484, 'solid': 3.0769231, 'cri': 2.9615386, 'tale': 2.92, 'surprisingli': 2.9, 'angel': 2.8666666, 'sweet': 2.857143, 'innoc': 2.8, 'sky': 2.7692308, 'happi': 2.764706, 'funniest': 2.75, 'rare': 2.75, 'crimin': 2.75, 'peter': 2.7142856, 'favourit': 2.7, 'william': 2.6923077, 'dirti': 2.642857, 'seven': 2.6363637, 'south': 2.6190476, 'fate': 2.6153846, 'boss': 2.6153846, 'romant': 2.6086957, 'definit': 2.607143, 'spi': 2.6, 'father': 2.596774, 'appreci': 2.58

In [ ]:
LR_map = dict(reversed(sorted(LR_map.items(), key=compare)))
print(f'{list(LR_map.items())[:5]}')
LR_map = dict(sorted(LR_map.items(), key=compare))
print(f'{list(LR_map.items())[:5]}')

[('perfectli', 5.5833335), ('strong', 5.2222223), ('henri', 4.6666665), ('harri', 4.6153846), ('delight', 4.5833335)]
[('stupid', 0.09625668), ('worst', 0.112068966), ('wast', 0.13496932), ('aw', 0.14925373), ('ridicul', 0.1495327)]


In [ ]:
positive_map = dict(reversed(sorted(positive_map.items(), key=compare)))
negative_map = dict(reversed(sorted(negative_map.items(), key=compare)))



In [ ]:
br = 0
for i, a in enumerate(positive_map.keys()):
    if i > 4:
        break
    br += 1
    print(a)

print('**************')
br = 0
for i, a in enumerate(negative_map.keys()):
    if i > 4:
        break
    br += 1
    print(a)



    # Reci dobijene pronalaskom najkoriscenijih iz oba skupa:

    # Pozitivne: br, film, /><, movi, /> Negativne: br movi />< /> film
    


    #Reci dobijene koriscenjem LR(rec) metrike:

    # [('perfectli', 5.5833335), ('strong', 5.2222223), ('henri', 4.6666665), ('harri', 4.6153846), ('delight', 4.5833335)]
    # [('stupid', 0.09625668), ('worst', 0.112068966), ('wast', 0.13496932), ('aw', 0.14925373), ('ridicul', 0.1495327)]

    # Na ovaj nacin postizemo da reci koje se cesto pojavljuju i u negativnim i
    # u pozitivnim komentarima izbegnemo, nalazimo samo reci koje se cesto 
    # pojavljuju u jednoj klasi, a znatno redje u drugoj




br
film
/><
movi
/>
**************
br
movi
/><
/>
film
